In [14]:
import os
import sys
import h5py
import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math
import tqdm
import random

import multiprocessing
from torchvision.utils import make_grid
from PIL import Image
import ast
import torchvision
from torchvision.datasets import ImageFolder

import requests
from PIL import ImageDraw
from PIL import ImageFont
from io import BytesIO
import IPython.display
from IPython.display import HTML
import matplotlib.pyplot as plt
from shutil import copyfile
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [15]:
from mobilenet_classifier import MobileNet
from resnet_classifier import ResNet

# Important! Choose the model and classifier you want to use here

In [16]:
model_names = {"plant", "faces_old", "faces_new"}
model_to_use = "plant"

In [17]:
stylex_path = None
classifier_name = None
data = None
USE_OLD_ARCHITECTURE = False

if model_to_use == "plant":
    stylex_path = "models/old_plant_mobilenet/model_260.pt"
    data = "../data/plant_village/all" # Plant dataset
    classifier_name = "mobilenet-64px-plant.pt" # Only use mobilenet for plants
    USE_OLD_ARCHITECTURE = True

elif model_to_use == "faces_old":
    stylex_path = "models/old_faces_gender_mobilenet/model_134.pt"
    data = "../data/Kaggle_FFHQ_Resized_256px/flickrfaceshq-dataset-nvidia-resized-256px/resized" # FFHQ faces dataset
    classifier_name = "resnet-18-64px-gender.pt"  # Use ResNet for all the gender related ones, even the one trained on mobilenet
    USE_OLD_ARCHITECTURE = True

elif model_to_use == "faces_new":
    stylex_path = "models/new_faces_gender_resnet/model_300.pt"
    data = "../data/Kaggle_FFHQ_Resized_256px/flickrfaceshq-dataset-nvidia-resized-256px/resized" # FFHQ faces dataset
    classifier_name = "resnet-18-64px-gender.pt"  # Use ResNet for all the gender related ones, even the one trained on mobilenet
    USE_OLD_ARCHITECTURE = False


if USE_OLD_ARCHITECTURE:
    from stylex_train import StylEx, Dataset, DistributedSampler, MNIST_1vA
    from stylex_train import image_noise, styles_def_to_tensor, make_weights_for_balanced_classes, cycle, default
else:
    from stylex_train_new import StylEx, Dataset, DistributedSampler, MNIST_1vA
    from stylex_train_new import image_noise, styles_def_to_tensor, make_weights_for_balanced_classes, cycle, default

In [18]:
NUM_CORES = multiprocessing.cpu_count()

@torch.no_grad()
def find_discriminator_threshold(stylex,
                                 classifier,
                                 dataloader,
                                 num_images,
                                 threshold_folder,
                                 dataset_name,
                                 image_size,
                                 batch_size,
                                 cuda_rank):

    d = h5py.File(os.path.join(threshold_folder, 'discriminator_threshold.hdf5'), 'w')
    discriminator_results = d.create_dataset("discriminator_outputs", (num_images, 1), dtype='f')
    generated_images = d.create_dataset("generated_images", (num_images, 3, image_size, image_size), dtype='f')
    
    print("Extracting discriminator results")
    for image_index in tqdm.tqdm(range(num_images)):
        image = next(dataloader)
        image = image.cuda(cuda_rank)
            
        image_logits = classifier.classify_images(image)
        w = stylex.encoder(image).unsqueeze(0)

        latent_w = None

        if USE_OLD_ARCHITECTURE:
            latent_w = [(torch.cat((w, image_logits), dim=1), stylex.G.num_layers)]
        else:
            latent_w = [(torch.cat((w, F.softmax(image_logits, dim=1)), dim=1), stylex.G.num_layers)]

        w_latent_tensor = styles_def_to_tensor(latent_w)
        image_generated, style_coords = stylex.G(w_latent_tensor, noise, get_style_coords=True)

        discriminator_output = None

        if USE_OLD_ARCHITECTURE:
            discriminator_output = discriminator_filter(stylex.D, image_generated, None)
        else:
            discriminator_output = discriminator_filter(stylex.D, image_generated, None, probabilities=F.softmax(classifier.classify_images(image_generated), dim=1))

        discriminator_results[image_index] = discriminator_output.cpu()
        generated_images[image_index] = image_generated.cpu()
        
    d.close()

# TODO: Upscaling
def plot_image(tensor) -> None:
    grid = make_grid(tensor,nrow=5)
    ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
    im = Image.fromarray(ndarr)
    display(im)
    
    
def load_hdf5_results(data_file, name, threshold):
    return np.array(data_file[name])[0:threshold]


def set_data_src(folder='./', dataset_name=None, image_size=64, batch_size=1, num_workers=4, is_ddp=False, rank=0, world_size=1):
    if dataset_name is None:
        dataset = Dataset(folder, image_size)
        num_workers = default(num_workers, NUM_CORES if not is_ddp else 0)

        sampler = DistributedSampler(dataset, rank=rank, num_replicas=world_size,
                                     shuffle=True) if is_ddp else None

        dataloader = DataLoader(dataset, num_workers=num_workers,
                                     batch_size=math.ceil(batch_size / world_size), sampler=sampler,
                                     shuffle=False, drop_last=True, pin_memory=True)

    elif dataset_name == 'MNIST':
        dataset = MNIST_1vA(digit=8)
        dataloader = DataLoader(dataset, batch_size=batch_size)

    else:
        raise NotImplementedError("This dataset is not supported yet. Please use dataset_name = None.")

    loader = cycle(dataloader)
    return dataset, loader


def sindex_to_block_idx_and_index(generator, sindex):
    tmp_idx = sindex

    block_idx = None
    idx = None

    for idx, block in enumerate(generator.blocks):
        if tmp_idx < block.num_style_coords:
            block_idx = idx
            idx = tmp_idx
            break
        else:
            tmp_idx = tmp_idx - block.num_style_coords

    return block_idx, idx


def get_min_max_style_vectors(style_coordinates):
    minimums = None
    maximums = None

    for style_coords in style_coordinates:
        if minimums is None or maximums is None:
            minimums = style_coords
            maximums = style_coords
        else:
            minimums = torch.minimum(minimums, style_coords)
            maximums = torch.maximum(maximums, style_coords)
    
    if minimums == None:
        raise ValueError('No images pass the threshold check')

    return minimums, maximums


def discriminator_filter(discriminator, generated_image, threshold, probabilities=None):
    if probabilities is not None:
        output_generated = discriminator(generated_image, probabilities=probabilities)
    else:
        output_generated = discriminator(generated_image)
    
    if threshold == None:
        return output_generated
    if output_generated < threshold:
        return (False, output_generated)
    else:
        return (True, output_generated)
    

def attfind_extraction(dataloader,
                       num_images,
                       results_folder,
                       stylex,
                       classifier,
                       dataset_name,
                       noise,
                       num_style_coords,
                       shift_size,
                       discriminator_threshold,
                       image_size=64,
                       batch_size=1,
                       cuda_rank = 0,
                       use_discriminator=False):
    
    if batch_size != 1:
        raise ValueError('Please use a batch_size equal to 1')

    with torch.no_grad():
        style_change_effects = torch.zeros((num_images, 2, num_style_coords, 2)).cuda(cuda_rank)
        image_latents = torch.zeros((num_images, 514)).cuda(cuda_rank)
        generated_image_classifications = torch.zeros((num_images, 2)).cuda(cuda_rank)
        style_coordinates = torch.zeros((num_images, num_style_coords)).cuda(cuda_rank)
        original_images = torch.zeros((num_images, 3, image_size, image_size)).cuda(cuda_rank)
        discriminator_results = torch.zeros((num_images, 1)).cuda(cuda_rank)
        
        # Select the images we want to use if use_discriminator is activated
        images_found = 0

        dataloader = iter(dataloader)

        for batch in dataloader:
            print(f"\rFound {images_found}/{num_images} images".format(images_found), end="")
            if images_found >= num_images:
                break

            batch = batch.cuda(cuda_rank)
            encoder_output = stylex.encoder(batch).unsqueeze(0)
            real_classified_logits = classifier.classify_images(batch)

            concat_w_tensor = None

            if USE_OLD_ARCHITECTURE:
                concat_w_tensor = torch.cat((encoder_output, real_classified_logits), dim=1)
            else:
                concat_w_tensor = torch.cat((encoder_output, F.softmax(real_classified_logits, dim=1)), dim=1)

            latent_w = [(concat_w_tensor, stylex.G.num_layers)]
            w_latent_tensor = styles_def_to_tensor(latent_w)
            generated_image, style_coords = stylex.G(w_latent_tensor, noise, get_style_coords=True)

            skip, discriminator_output = None, None

            if USE_OLD_ARCHITECTURE:
                skip, discriminator_output = discriminator_filter(stylex.D, generated_image, discriminator_threshold)
            else:
                skip, discriminator_output = discriminator_filter(stylex.D, generated_image, discriminator_threshold, probabilities=F.softmax(classifier.classify_images(generated_image), dim=1))
            
            if use_discriminator and skip:
                continue
            else:
                original_images[images_found] = batch
                image_latents[images_found] = concat_w_tensor
                style_coordinates[images_found] = style_coords
                discriminator_results[images_found] = discriminator_output
                generated_image_classifications[images_found] = classifier.classify_images(generated_image)

                images_found += 1

        print()
        print("Retrieving min and max")
        minima, maxima = get_min_max_style_vectors(style_coordinates)
        
        print("Exploring StyleSpace attributes")
        
        #for filtered_image in filtered_images:  #TODO: Change this to a for loop
        image_index = 0
        for image_index in range(images_found):
            style_coords = style_coordinates[image_index]
            image_generated_logits = generated_image_classifications[image_index]

            concat_w_tensor = image_latents[image_index].unsqueeze(0)
            latent_w = [(concat_w_tensor, stylex.G.num_layers)]
            w_latent_tensor = styles_def_to_tensor(latent_w)

            style_change_effect = torch.Tensor(1, 2, num_style_coords, 2).cuda(cuda_rank)

            for sindex in tqdm.tqdm(range(num_style_coords)):

                block_idx, weight_idx = sindex_to_block_idx_and_index(stylex.G, sindex)
                block = stylex.G.blocks[block_idx]

                current_style_layer = None
                one_hot = None

                if weight_idx < block.input_channels:
                    current_style_layer = block.to_style1
                    one_hot = torch.zeros((1, block.input_channels)).cuda(cuda_rank)
                else:
                    weight_idx -= block.input_channels
                    current_style_layer = block.to_style2
                    one_hot = torch.zeros((1, block.filters)).cuda(cuda_rank)

                one_hot[:, weight_idx] = 1

                s_shift_down = one_hot * ((minima[sindex] - style_coords[sindex]) * shift_size)
                s_shift_up = one_hot * ((maxima[sindex] - style_coords[sindex]) * shift_size)

                for direction_index, shift in enumerate([s_shift_down, s_shift_up]):

                    shift = shift.squeeze(0)

                    current_style_layer.bias += shift
                    perturbed_generated_images = stylex.G(w_latent_tensor, noise)

                    shift_logits = classifier.classify_images(perturbed_generated_images)
                    style_change_effect[0, direction_index, sindex] = shift_logits - image_generated_logits
            
                    current_style_layer.bias -= shift

            style_change_effects[image_index] = style_change_effect
            image_index += 1

        
        print("Initializing file")
        f = h5py.File(os.path.join(results_folder, 'style_change_records.hdf5'), 'w')
        file_style_change_effects = f.create_dataset('style_change', (num_images, 2, num_style_coords, 2), dtype='f')
        file_image_latents = f.create_dataset('latents', (num_images, 514), dtype='f')
        file_generated_image_classifications = f.create_dataset('base_prob', (num_images, 2), dtype='f')
        file_save_minima = f.create_dataset("minima", (1, num_style_coords), dtype='f')
        file_save_maxima = f.create_dataset("maxima", (1, num_style_coords), dtype='f')
        file_style_coordinates = f.create_dataset("style_coordinates", (num_images, num_style_coords), dtype='f')
        file_original_images = f.create_dataset("original_images", (num_images, 3, image_size, image_size), dtype='f')
        file_save_noise = f.create_dataset("noise", (1, image_size, image_size, 1), dtype='f')
        file_discriminator_results = f.create_dataset("discriminator", (num_images, 1), dtype='f')

        file_style_change_effects[:] = style_change_effects.cpu()
        file_image_latents[:] = image_latents.cpu()
        file_generated_image_classifications[:] = generated_image_classifications.cpu()
        file_style_coordinates[:] = style_coordinates.cpu()
        file_original_images[:] = original_images.cpu()
        file_discriminator_results[:] = discriminator_results.cpu()
        
        
        file_save_noise[0] = noise.cpu()
        file_save_minima[0] = minima.cpu()
        file_save_maxima[0] = maxima.cpu()
                
        f.close()


In [19]:
def model_loader(stylex_path,
                   classifier_name,
                   image_size,
                   cuda_rank):

    init_stylex = StylEx(image_size=image_size)
    init_stylex.load_state_dict(torch.load(stylex_path)["StylEx"])

    init_classifier = None

    if "mobilenet" in classifier_name.lower():
        init_classifier = MobileNet(classifier_name, cuda_rank=cuda_rank, output_size=2, image_size=image_size)
    elif "resnet" in classifier_name.lower():
        init_classifier = ResNet(classifier_name, cuda_rank=cuda_rank, output_size=2, image_size=image_size)
    else:
        raise NotImplementedError("This classifier is not supported yet, please add support or change the filename to contain MobileNet or ResNet.")
    return init_stylex, init_classifier

In [20]:
results_folder = './'
threshold_folder = './'
dataset_name = None         # for any dataset that is not MNIST

num_style_coords = 2464     # TODO: Still hard coded since I didnt know how to retrieve this from the model
image_size = 64             # Depends on dataset
cuda_rank = 0
num_images = 10            # Will take approximately 5 hours for 250 image of size 64x64
shift_size = 1            # Magnitude of the step taken in the stylespace
batch_size = 1              # Code currently only supports a batch_size of 1
noise = image_noise(batch_size, image_size, device=cuda_rank)
use_discriminator = False
discriminator_threshold = -0.5 # specific for plants

In [21]:
dataset, dataloader = set_data_src(folder=data, dataset_name=dataset_name, image_size=image_size, batch_size=batch_size)

In [22]:
stylex, classifier = model_loader(stylex_path = stylex_path,
                                  classifier_name = classifier_name,
                                  image_size = image_size,
                                  cuda_rank = cuda_rank)

Using cache found in C:\Users\noahv/.cache\torch\hub\pytorch_vision_v0.10.0


In [23]:
# Careful! Running this will overwrite the saved stylespace exploration file.
try:
    os.remove("./style_change_records.hdf5")
except:
    pass

attfind_extraction(dataloader = dataloader,
                   num_images = num_images,
                   results_folder = results_folder,
                   dataset_name = dataset_name,
                   stylex = stylex,
                   classifier = classifier,
                   noise=noise,
                   num_style_coords = num_style_coords,
                   shift_size = shift_size,
                   batch_size = batch_size,
                   use_discriminator = use_discriminator,
                   discriminator_threshold = discriminator_threshold)

Found 10/10 images
Retrieving min and max
Exploring StyleSpace attributes


100%|██████████| 2464/2464 [00:50<00:00, 49.03it/s]


Initializing file


OSError: Unable to create file (unable to truncate a file which is already open)

In [ ]:
def filter_unstable_images(style_change_effect,
                           effect_threshold = 0.3,
                           num_indices_threshold = 150):

    unstable_images = (np.sum(np.abs(style_change_effect) > effect_threshold, axis=(1, 2, 3)) > num_indices_threshold)
    style_change_effect[unstable_images] = 0
    
    return style_change_effect

In [ ]:
threshold_index = 501

hf = h5py.File('./style_change_records.hdf5', 'r')

style_change_effect = load_hdf5_results(hf, "style_change", threshold_index)
W_values = load_hdf5_results(hf, "latents", threshold_index)
base_probs = load_hdf5_results(hf, "base_prob", threshold_index)
all_style_vectors = load_hdf5_results(hf, "style_coordinates", threshold_index)
original_images = load_hdf5_results(hf, "original_images", threshold_index)
discriminator_results = load_hdf5_results(hf, "discriminator", threshold_index)

saved_noise = np.array(hf["noise"])
style_min = np.squeeze(np.array(hf["minima"]))
style_max = np.squeeze(np.array(hf["maxima"]))

#style_change_effect = filter_unstable_images(style_change_effect)
saved_noise = torch.Tensor(saved_noise).cuda(cuda_rank)

all_style_vectors_distances = np.zeros((all_style_vectors.shape[0], all_style_vectors.shape[1], 2))
all_style_vectors_distances[:,:, 0] = all_style_vectors - np.tile(style_min, (all_style_vectors.shape[0], 1))
all_style_vectors_distances[:,:, 1] = np.tile(style_max, (all_style_vectors.shape[0], 1)) - all_style_vectors

If the next code block gives an "arrays used as indices must be of integer (or boolean) type" you might want to run with more images.

In [ ]:
all_labels = np.argmax(base_probs, axis=1)
style_effect_classes = {}
W_classes = {}
style_vectors_distances_classes = {}
all_style_vectors_classes = {}

for img_ind in range(2):
    img_inx = np.array([i for i in range(all_labels.shape[0]) if all_labels[i] == img_ind])
    curr_style_effect = np.zeros((len(img_inx), style_change_effect.shape[1], style_change_effect.shape[2], style_change_effect.shape[3]))
    curr_w = np.zeros((len(img_inx), W_values.shape[1]))
    curr_style_vector_distances = np.zeros((len(img_inx), style_change_effect.shape[2], 2))
    for k, i in enumerate(img_inx):
        curr_style_effect[k, :, :] = style_change_effect[i, :, :, :]
        curr_w[k, :] = W_values[i, :]
        curr_style_vector_distances[k, :, :] = all_style_vectors_distances[i, :, :]
    style_effect_classes[img_ind] = curr_style_effect
    W_classes[img_ind] = curr_w
    style_vectors_distances_classes[img_ind] = curr_style_vector_distances
    all_style_vectors_classes[img_ind] = all_style_vectors[img_inx]
    print(f'Class {img_ind}, {len(img_inx)} images.')

In [ ]:
def find_significant_styles(style_change_effect,
                            num_indices,
                            class_index,
                            generator,
                            classifier,
                            all_dlatents,
                            style_min,
                            style_max,
                            max_image_effect = 0.2,
                            label_size = 2,
                            sindex_offset = 0):
  
    num_images = style_change_effect.shape[0]
    #print(class_index)
    style_effect_direction = np.maximum(0, style_change_effect[:, :, :, class_index].reshape((num_images, -1)))

    images_effect = np.zeros(num_images)
    all_sindices = []
    discriminator_removed = []

    while len(all_sindices) < num_indices:
        next_s = np.argmax(np.mean(style_effect_direction[images_effect < max_image_effect], axis=0))

        all_sindices.append(next_s)
        images_effect += style_effect_direction[:, next_s]
        style_effect_direction[:, next_s] = 0

    return [(x // style_change_effect.shape[2], (x % style_change_effect.shape[2]) + sindex_offset) for x in all_sindices]

In [ ]:
label_size_clasifier = 2
num_indices =  5
effect_threshold = 0.5
s_indices_and_signs_dict = {}

for class_index in [0, 1]:
    #split_ind = 1 - class_index
    split_ind = class_index
    all_s = style_effect_classes[split_ind]
    all_w = W_classes[split_ind]

    # Find s indicies
    s_indices_and_signs = find_significant_styles(style_change_effect=all_s,
                                                  num_indices=num_indices,
                                                  class_index=class_index,
                                                  generator=stylex.G,
                                                  classifier=classifier,
                                                  all_dlatents=all_w,
                                                  style_min=style_min,
                                                  style_max=style_max,
                                                  max_image_effect=effect_threshold*5,
                                                  label_size=label_size_clasifier,
                                                  sindex_offset=0)

    s_indices_and_signs_dict[class_index] = s_indices_and_signs

sindex_class_0 = [sindex for _, sindex in s_indices_and_signs_dict[0]]
all_sindex_joined_class_0 = [(1 - direction, sindex) for direction, sindex in s_indices_and_signs_dict[1] if sindex not in sindex_class_0]
all_sindex_joined_class_0 += s_indices_and_signs_dict[0]
scores = []

for direction, sindex in all_sindex_joined_class_0:
    other_direction = 1 if direction == 0 else 0
    curr_score = np.mean(style_change_effect[:, direction, sindex, 0]) + np.mean(style_change_effect[:, other_direction, sindex, 1])
    scores.append(curr_score)

s_indices_and_signs = [all_sindex_joined_class_0[i] for i in np.argsort(scores)[::-1]]

print('Directions and style indices for moving from class 1 to class 0 = ', s_indices_and_signs[:num_indices])
print('Use the other direction to move for class 0 to 1.')

In [ ]:
def generate_change_image_given_dlatent(dlatent,
                                        generator,
                                        classifier,
                                        class_index,
                                        sindex,
                                        s_style_min,
                                        s_style_max,
                                        style_direction_index,
                                        shift_size,
                                        label_size,
                                        noise, 
                                        cuda_rank):

    w_latent_tensor = styles_def_to_tensor(dlatent)
    image_generated, style_coords = generator(w_latent_tensor, noise, get_style_coords=True)

    block_idx, weight_idx = sindex_to_block_idx_and_index(generator, sindex)
    block = generator.blocks[block_idx]

    current_style_layer = None
    one_hot = None

    if weight_idx < block.input_channels:
        current_style_layer = block.to_style1
        one_hot = torch.zeros((1, block.input_channels)).cuda(cuda_rank)
    else:
        weight_idx -= block.input_channels
        current_style_layer = block.to_style2
        one_hot = torch.zeros((1, block.filters)).cuda(cuda_rank)

    one_hot[:, weight_idx] = 1

    if style_direction_index == 0:
        shift = one_hot * ((s_style_min - style_coords[:, sindex]) * shift_size).unsqueeze(1)
    else:
        shift = one_hot * ((s_style_max - style_coords[:, sindex]) * shift_size).unsqueeze(1)

    with torch.no_grad():
        shift = shift.squeeze(0)
        current_style_layer.bias += shift
        perturbed_generated_images, style_coords = generator(w_latent_tensor, noise, get_style_coords=True)
        shift_logits = classifier.classify_images(perturbed_generated_images)
        change_prob = torch.softmax(shift_logits, dim=1).cpu().detach().numpy()[0, class_index]
        current_style_layer.bias -= shift

    return perturbed_generated_images, change_prob

In [ ]:
def draw_on_image(image,
                  number,
                  font_file,
                  font_fill = (0, 0, 255)):

    image = np.transpose(image, (1, 2, 0))
    image = np.clip(image, 0, 1)
    fnt = ImageFont.truetype(font_file, 20)
    out_image = Image.fromarray((image * 255).astype(np.uint8))
    draw = ImageDraw.Draw(out_image)
    #draw.multiline_text((10, 10), ('%.3f' % number), font=fnt, fill=font_fill)
    return np.array(out_image)

In [ ]:
def generate_images_given_dlatent(dlatent,
                                  generator,
                                  classifier,
                                  class_index,
                                  sindex,
                                  s_style_min,
                                  s_style_max,
                                  style_direction_index,
                                  font_file,
                                  noise,
                                  shift_size = 2,
                                  label_size = 2,
                                  draw_results_on_image = True,
                                  resolution = 64,
                                  cuda_rank = 0,
                                  gen_num_layers = 5):
  
    result_image = np.zeros((resolution, 2 * resolution, 3), np.uint8)
    dlatent = [(torch.Tensor(dlatent).cuda(cuda_rank), gen_num_layers)]
    w_latent_tensor = styles_def_to_tensor(dlatent)
    base_image, style_coords = generator(w_latent_tensor, noise, get_style_coords=True)
    result = classifier.classify_images(base_image)
    base_prob = torch.softmax(result, dim=1).cpu().detach().numpy()[0, class_index]

    if draw_results_on_image:
        result_image[:, :resolution, :] = draw_on_image(base_image[0].cpu().detach().numpy(), base_prob, font_file)
    else:
        result_image[:, :resolution, :] = (base_image[0].cpu().detach().numpy() * 127.5 + 127.5).astype(np.uint8)

    change_image, change_prob = (generate_change_image_given_dlatent(dlatent, generator, classifier,
                                                                     class_index, sindex,
                                                                     s_style_min, s_style_max,
                                                                     style_direction_index, shift_size,
                                                                     label_size, noise=noise, cuda_rank=0))
 
    if draw_results_on_image:
        result_image[:, resolution:, :] = draw_on_image(change_image[0].cpu().detach().numpy(), change_prob, font_file)

    else:
        result_image[:, resolution:, :] = (np.maxiumum(np.minimum(change_image[0].cpu().detach().numpy(), 1), -1) * 127.5 + 127.5).astype(np.uint8)

    return (result_image, change_prob, base_prob)

In [ ]:
def visualize_style(generator,
                    classifier,
                    all_dlatents,
                    style_change_effect,
                    style_min,
                    style_max,
                    sindex,
                    style_direction_index,
                    max_images,
                    shift_size,
                    font_file,
                    noise,
                    label_size = 2,
                    class_index = 0,
                    effect_threshold = 0.3,
                    seed = None,
                    allow_both_directions_change = False,
                    draw_results_on_image = True):
  
    if allow_both_directions_change:
        images_idx = (np.abs(style_change_effect[:, style_direction_index, sindex,
                                             class_index]) >
                  effect_threshold).nonzero()[0]
    else:
        images_idx = ((style_change_effect[:, style_direction_index, sindex,
                                       class_index]) >
                  effect_threshold).nonzero()[0]
    if images_idx.size == 0:
        return np.array([])

    if seed is not None:
        np.random.seed(seed)
    np.random.shuffle(images_idx)
    images_idx = images_idx[:min(max_images*10, len(images_idx))]
    dlatents = all_dlatents[images_idx]

    result_images = []
    for i in range(len(images_idx)):
        cur_dlatent = dlatents[i:i + 1]
        (result_image, base_prob, change_prob) = generate_images_given_dlatent(
                                                 dlatent=cur_dlatent,
                                                 generator=generator,
                                                 classifier=classifier,
                                                 class_index=class_index,
                                                 sindex=sindex,
                                                 noise=noise,
                                                 s_style_min=style_min[sindex],
                                                 s_style_max=style_max[sindex],
                                                 style_direction_index=style_direction_index,
                                                 font_file=font_file,
                                                 shift_size=shift_size,
                                                 label_size=label_size,
                                                 draw_results_on_image=draw_results_on_image,
                                                 gen_num_layers=stylex.G.num_layers)

        if np.abs(change_prob - base_prob) < effect_threshold:
            continue
        result_images.append(result_image)
        if len(result_images) == max_images:
            break

    if len(result_images) < 3:
        # No point in returning results with very little images
        return np.array([])
    return np.concatenate(result_images[:max_images], axis=0)

In [ ]:
def visualize_style_by_distance_in_s(generator,
                                    classifier,
                                    all_dlatents,
                                    all_style_vectors_distances,
                                    style_min,
                                    style_max,
                                    sindex,
                                    style_sign_index,
                                    max_images,
                                    shift_size,
                                    font_file,
                                    noise,
                                    label_size = 2,
                                    class_index = 0,
                                    draw_results_on_image = True,
                                    effect_threshold = 0.1, 
                                    cuda_rank=0):

    images_idx = np.argsort(all_style_vectors_distances[:, sindex, style_sign_index])[::-1]

    if images_idx.size == 0:
        print("images_idx size is zero")
        return np.array([])

    images_idx = images_idx[:min(max_images*10, len(images_idx))]
    dlatents = all_dlatents[images_idx]

    result_images = []
    for i in range(len(images_idx)):
        cur_dlatent = dlatents[i:i + 1]
        (result_image, change_prob, base_prob) = generate_images_given_dlatent(dlatent=cur_dlatent,
                                                                                 generator=generator,
                                                                                 classifier=classifier,
                                                                                 class_index=class_index,
                                                                                 sindex=sindex,
                                                                                 noise=noise,
                                                                                 s_style_min=style_min[sindex],
                                                                                 s_style_max=style_max[sindex],
                                                                                 style_direction_index=style_sign_index,
                                                                                 font_file=font_file,
                                                                                 shift_size=shift_size,
                                                                                 label_size=label_size,
                                                                                 draw_results_on_image=draw_results_on_image,
                                                                                 cuda_rank = 0)
        result_images.append(result_image)

    if len(result_images) < 3:
        return np.array([])
    return np.concatenate(result_images[:max_images], axis=0)


In [ ]:
def show_image(image, fmt='png'):

    if image.dtype == np.float32:
        image = np.uint8(image * 255)

    if image.shape[0] == 3:
        image = np.transpose(image, (1, 2, 0))

    plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(image)
    plt.show()

In [ ]:
max_images = 10
sindex = 2449    # Put the stylespace indices from the print statement above here, together with the sign corresponding to the direction.
class_index = 0  # The class we want to change
#shift_sign = "0"
wsign_index = 0
label_size = 2

shift_size = 2
effect_threshold =  0.5
split_by_class = True
select_images_by_s_distance = True
draw_results_on_image = True 

if split_by_class:
    split_ind = 1 if class_index == 1 else 0 #1 if class_index == 0 else 0
    all_s = style_effect_classes[split_ind]
    all_w = W_classes[split_ind]
    all_s_distances = style_vectors_distances_classes[split_ind]
else:
    all_s = style_change_effect
    all_w = W_values
    all_s_distances = all_style_vectors_distances

font_file = './arialuni.ttf'
if not os.path.exists(font_file):
    r = requests.get('https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/ipwn/arialuni.ttf')
    open(font_file, 'wb').write(r.content)

if not select_images_by_s_distance:
    yy = visualize_style(stylex.G,
                         classifier,
                         all_w,
                         all_s,
                         style_min,
                         style_max,
                         sindex,
                         wsign_index,
                         max_images=max_images,
                         shift_size=shift_size,
                         font_file=font_file,
                         label_size=label_size,
                         class_index=class_index,
                         effect_threshold=effect_threshold,
                         draw_results_on_image=draw_results_on_image,
                         noise = saved_noise)
    
else:
    yy = visualize_style_by_distance_in_s(stylex.G,
                                          classifier,
                                          all_w,
                                          all_s_distances,
                                          style_min,
                                          style_max,
                                          sindex,
                                          wsign_index,
                                          max_images=max_images,
                                          shift_size=shift_size,
                                          font_file=font_file,
                                          label_size=label_size,
                                          class_index=class_index,
                                          effect_threshold=effect_threshold,
                                          draw_results_on_image=draw_results_on_image, cuda_rank=0,
                                          noise = saved_noise)

if yy.size > 0:
    show_image(yy)
else:
    print('no images found')